This is an implementation of the Grover's algorithm. 

To write the code, I mostly followed this example from IBM's Quiskit github (https://github.com/Qiskit/textbook/blob/main/notebooks/ch-algorithms/grover.ipynb). I make comments along the way to explain the process and the circuit that we will be building.

The purpose of Grover's algorithm is to solve unstructured search problems. Such a problem can be for example trying to find the combination (referred to as the winner state |<i>w</i>>) that unlocks a lock. It might not be easy to find the correct combination |<i>w</i>> among all possible combinations N, but it is fairly simple to validate a given combination and see if it is correct or not. This algorithm requires on average N/2 (O(N)) computations on a classical computer, as each of the N combinations needs to be tested one at a time. On the other hand, Grover's algorithm uses the fact that we can manipulate many qubits' superpositions at the same time to speed up the process to O(<span>&radic;N</span>).

An overview of the Grovers algorithm:
1. State preparation
2. Oracle step (Solution marking)
3. Diffusion operation (Amplitude amplification)

In our example, we create a simple version of Grover's algorithm for N=4 in order to demonstrate how it works and how the circuit is coded. I ran this on IBM's Quantum lab platform

First lets import the necessary libraries:

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import math
from qiskit import IBMQ, Aer, transpile, execute
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit.providers.ibmq import least_busy
from qiskit.visualization import plot_histogram

print('Done')

1. State preparation
--------------------

Here we will create our qubits and apply a Hadamard gate to them in order to set them from their initial states (usually |0>) to uniform superposition. For one qubit, we go from the |0> state to the |+> state, for two qubits we go to from |00> to &frac12;(|00>+|01>+|10>+|11>). This means that if we measure our system after the Hadarmard gate, the probability of getting any one of the combinations will be the same. Our winner state |w> is one of the combinations (|00>,|01>,|10>,|11>) in our state <i>s</i>

In [ ]:
number_of_qubits=2
circuit = QuantumCircuit(number_of_qubits)

def state_prep(circ, qubits):
    for q in qubits:
        circ.h(q)
    return circ

circuit = state_prep(circuit, [0,1])
circuit.draw()

Output:

<img src="Output1.png" alt="Output1">

2. Oracle creation
------------------

Here we will create a simple oracle, where we already know the solution to our search. The oracle (commonly denoted as <i>Uw</i>) is the key operation in Grover's algorithm that reveals/marks the solutions. It does this by adding a negative phase to the solutions of the algorithm. In our example, we will make a simple version of this process by using a controlled Z gate, that will add a negative phase to the |11> state within our state |<i>s</i>>.

To apply Grover's algorithm to more complicated problems, we would need to create a custom oracle, by tranlating a classical function into a quantum circuit, which will check each combination against the function and give a negative phase to that part of our state (combination) which satisfies the function. 

In [ ]:
circuit.cz(0,1)
circuit.draw()

Output:

<img src="Output2.png" alt="Output2">

3. Diffusion operation
----------------------

Now that we have 'marked' our winner state |w>, we need to make sure that when we measure our total state |s>, we will get |w>, which is still only a small part of our total state, and the probability of measuring it is just as high as any other non-winner state (typicall referred to as |s'>). To do this, we need to somehow amplify the amplitute of |w>. This can be done by performing a reflection around state |s>, the unitery operation for which is called Us: Us = 2|s>\<s| - 1

This operation can be done using a combination of Hadamard and Z gates, as can be seen below.

In [ ]:
circuit.h([0,1])
circuit.z([0,1])
circuit.cz(0,1)
circuit.h([0,1])
circuit.draw()

Output:

<img src="Output3.png" alt="Output3">

**Running the circuit in a Real Quantum Computer (through IBM lab)**

We first find an available device

In [ ]:
provider = IBMQ.load_account()
provider = IBMQ.get_provider("ibm-q")
device = least_busy(provider.backends(filters=lambda x: x.configuration().n_qubits >= 3 and 
                                   not x.configuration().simulator and x.status().operational==True))
print("Running on: ", device)

And then we wait for our turn and run our circuit in the device

In [ ]:
from qiskit.tools.monitor import job_monitor
transpiled_grover_circuit = transpile(circuit, device, optimization_level=3)
job = device.run(transpiled_grover_circuit)
job_monitor(job, interval=2)

Visualize the results

In [ ]:
results = job.result()
answer = results.get_counts(circuit)
plot_histogram(answer)

Output:

<img src="Output4.png" alt="Output3">

As we can see, state 11 (our marked state) was measured most of the time, with a slight error occuring sometimes, were we measure our winner to be 00, 01 or 10. Grover's algorithm becomes more precise if we repeat the oracle and amplitude amplification step, placing our state |s> closer to state |w> with each repetition and increasing the probability of the measured |w> to be correct.